# NTOU Machine learning HW1
## by 00857005 周固廷

In this exercise, we try to compare the accuracy of the two classifiers: k-NN and linear SVM on  the Breast Cancer Wisconsin (Diagnostic) dataset by nested cross-validation.  The following are the settings for this exercise.

1. Hyperparameters:
    The hyperparameter for k-NN is n_neighbors and the candidates for this hyperparameter are [1,3,5,7].
    The hyperparameter for linear SVM is C and the candidates for C are [0.01,0.1,1,10].
2. Nested cross-validation
   outer-loop: 10-fold stratified cross-validation
   inner-loop: GridSearchCV with cv=5
3. dataset
    import sklearn.datasets as ds
    data, target = ds.load_breast_cancer(True)

Useful links:
    + linear SVM (sklearn.svm.SVC): https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

    + k-nearest neighbor classifier (sklearn.neighbors.KNeighborsClassifier): https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

    + dataset: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer

In [10]:
import sklearn.datasets as ds

data, target = ds.load_breast_cancer(return_X_y = True)

In [11]:
data[0]

array([1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
       3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
       8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
       3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
       1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01])

In [13]:
from sklearn import svm

In [18]:
clf = svm.SVC()

In [20]:
clf.fit(data, target)

SVC()

In [21]:
clf.score(data, target)

0.9226713532513181

In [34]:
import numpy as np
from sklearn.model_selection import StratifiedKFold,cross_val_score,GridSearchCV
#cv代表cross validation
outer_kFold = StratifiedKFold(n_splits=10 , shuffle=True ,random_state=2)  #外層的分層抽樣 (拆成10fold)
inner_cv = GridSearchCV(clf, {'C':[0.01,0.1,1,10]}, cv=5)  # 內層的validation tuning (拆成5fold)
scores = cross_val_score(inner_cv, data, target, cv=outer_kFold)
# for train_index, test_index in kFold.split(data,target):
#     #每次拆分出Training和testing data
#      print("TRAIN:", train_index, "TEST:", test_index)
#      X_train, X_test = data[train_index], data[test_index]
#      y_train, y_test = target[train_index], target[test_index]

In [35]:
print(scores)

[0.87719298 0.92982456 0.89473684 0.92982456 0.87719298 0.9122807
 0.96491228 0.98245614 0.92982456 0.92857143]
